In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np

from dag_prf_utils.pyctx import PyctxSurf
from dag_prf_utils.utils import *
from dag_prf_utils.mesh_maker import *
from dag_prf_utils.prfpy_functions import *

from amb_scripts.load_saved_info import *
from amb_scripts.utils import *

import os
import time
from datetime import datetime, timedelta

# fs_dir = os.environ["SUBJECTS_DIR"]
fs_dir = '/data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/derivatives/freesurfer'
print(fs_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/derivatives/freesurfer


In [4]:
sub_list = ['sub-01', 'sub-02']
ses_list = ['ses-1', 'ses-2']


roi_fit = 'all'
fit_stage='iter'

model_list = ['gauss', 'csf']#, 'css', 'dog', 'norm']
prf_multi = {}

for sub in sub_list:
    prf_obj_list = [] 
    prf_id_list = []
    for i_ses,ses in enumerate(ses_list):        
        for eye in ['L', 'R']:    
            for model in model_list:
                if 'csf' in model:
                    task = f'CSF{eye}E'
                    # roi_fit = 'all'
                else:
                    task = f'pRF{eye}E'
                    # roi_fit = 'all'

                this_prf_data = amb_load_prf_params(
                    sub=sub,
                    task_list=task,
                    model_list=model,
                    roi_fit=roi_fit, 
                    fit_stage=fit_stage,
                    ses=ses
                )[task][model]
                                                        
                prf_obj_list.append(Prf1T1M(this_prf_data, model))
                prf_id_list.append(f'{eye}_{i_ses+1}_{model}')   
            
    prf_multi[sub] = PrfMulti(prf_obj_list, prf_id_list)

/data1/projects/dumoulinlab/Lab_members/Marcus/programs/dag_prf_utils/dag_prf_utils/prfpy_functions.py:308: RuntimeWarning: divide by zero encountered in log10
  self.params_dd['log10_sf0'] = np.log10(self.params_dd['sf0'])
/data1/projects/dumoulinlab/Lab_members/Marcus/programs/dag_prf_utils/dag_prf_utils/prfpy_functions.py:309: RuntimeWarning: divide by zero encountered in log10
  self.params_dd['log10_maxC'] = np.log10(self.params_dd['maxC'])
/data1/projects/dumoulinlab/Lab_members/Marcus/programs/dag_prf_utils/dag_prf_utils/prfpy_functions.py:311: RuntimeWarning: divide by zero encountered in true_divide
  10**(np.sqrt(self.params_dd['log10_maxC']/(self.params_dd['width_r']**2)) + \
/data1/projects/dumoulinlab/Lab_members/Marcus/programs/dag_prf_utils/dag_prf_utils/prfpy_functions.py:311: RuntimeWarning: invalid value encountered in sqrt
  10**(np.sqrt(self.params_dd['log10_maxC']/(self.params_dd['width_r']**2)) + \
/data1/projects/dumoulinlab/Lab_members/Marcus/programs/dag_prf_ut

In [9]:
# Add AREA UN LOG CSF
# load prfpy stim
prfpy_stim = amb_load_prfpy_stim('CSF')
m = 'csf'
for sub in sub_list:
    prf_obj_list = [] 
    prf_id_list = []
    for ses in ['1', '2']:        
        for eye in ['L', 'R']:    
            task = f'CSF{eye}E'            
            prf_multi[sub].prf_obj[f'{eye}_{ses}_{m}'].pd_params['AULCSF_sum'] = add_aulcsf_to_obj(
                prf_obj = prf_multi[sub].prf_obj[f'{eye}_{ses}_{m}'], 
                prfpy_stim = prfpy_stim)
            
            prf_multi[sub].prf_obj[f'{eye}_{ses}_{m}'].pd_params['AULCSF_trap'] = add_aulcsf_to_objV2(
                prf_obj = prf_multi[sub].prf_obj[f'{eye}_{ses}_{m}'], 
                prfpy_stim = prfpy_stim)            
#              

/data1/projects/dumoulinlab/Lab_members/Marcus/programs/prfpy/prfpy/stimulus.py:252: RuntimeWarning: divide by zero encountered in log10
  self.log_SF_seq = np.log10(SF_seq)
/data1/projects/dumoulinlab/Lab_members/Marcus/programs/prfpy/prfpy/stimulus.py:254: RuntimeWarning: divide by zero encountered in true_divide
  self.CON_S_seq = 100/CON_seq
/data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/code/amb_code/amb_scripts/utils.py:101: RuntimeWarning: divide by zero encountered in log10
  log_SFs = np.log10(SFs)
/data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/code/amb_code/amb_scripts/utils.py:102: RuntimeWarning: divide by zero encountered in log10
  log_sf0 = np.log10(sf0)
/data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/code/amb_code/amb_scripts/utils.py:103: RuntimeWarning: divide by zero encountered in log10
  log_maxC = np.log10(maxC)
/data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/code/amb_code/am

In [5]:
# Add CMF
m = 'gauss'
for sub in sub_list:
    prf_obj_list = [] 
    prf_id_list = []
    for ses in ['1', '2']:        
        for eye in ['L', 'R']:    
            task = f'pRF{eye}E'
            cmf = amb_load_cmf(sub,task,f'ses-{ses}')[task]
            prf_multi[sub].prf_obj[f'{eye}_{ses}_{m}'].pd_params['cmf'] = cmf

In [ ]:
sum(amb_load_nverts('sub-01'))

In [ ]:
prf_multi['sub-01'].scatter(
    px = 'L_1_csf-AULCSF_sum', 
    py = 'L_1_gauss-cmf',
    do_line=True,
    do_corr=True,
    th_plus={'L_1_gauss-min-cmf':-1}
)
# plt.gca().set_aspect('equal', adjustable='box')
plt.gcf().set_size_inches(10,10)

In [12]:
sub = 'sub-02'
fs = FSMaker(sub=sub,fs_dir=fs_dir)
m = 'csf'
p = 'AULCSF_trap'
vmin = 0
vmax = 1
for eye in ['L', 'R']:
    for ses in ['1', '2']:
        this_data = prf_multi[sub].prf_obj[f'{eye}_{ses}_{m}'].pd_params[p].to_numpy()
        vx_mask = prf_multi[sub].prf_obj[f'{eye}_{ses}_{m}'].return_vx_mask({'min-rsq':.1, })
        fs.add_surface(
            data = this_data,
            surf_name = f'{sub}-{eye}-{ses}-{m}-{p}',    
            # vmin = vmin, vmax=vmax, # min and max values of polar anlge 
            data_mask=vx_mask,
            cmap = 'magma', 
        )
fs.open_fs_surface(fs.surf_list)


Saving sub-02-L-1-csf-AULCSF_trap in /data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/derivatives/freesurfer/sub-02/surf/custom
Saving sub-02-L-2-csf-AULCSF_trap in /data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/derivatives/freesurfer/sub-02/surf/custom
Saving sub-02-R-1-csf-AULCSF_trap in /data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/derivatives/freesurfer/sub-02/surf/custom
Saving sub-02-R-2-csf-AULCSF_trap in /data1/projects/dumoulinlab/Lab_members/Marcus/projects/amblyopia_emc/derivatives/freesurfer/sub-02/surf/custom


In [ ]:
this_data.shape

In [ ]:
cprf_obj['L'].rapid_p_corr(
    'ecc', 'cmf', th={'min-rsq':.2, 'max-ecc':5, 'min-cmf':-1}, do_line=True, dot_alpha=0.1
)
plt.show()

In [ ]:
# Example for speeding up geodesic_distance
from dag_prf_utils.threading import *


# Ignore above - just another way to get multithreading working
def geodesic_distance_wrapper(ivx):
    # your surface here 
    return gdist.surf['lh'].geodesic_distance(ivx)

gdist_thread = DagThreader(io_function=geodesic_distance_wrapper)
total_nvx = 50 # test for 50 voxels
total_threads = 5 # run with 5 threads
result_fast = gdist_thread.run_threader(
    input_list = list(range(total_nvx)),
    max_size = total_threads)

# check it with the slow loop
result_slow = []
for i  in range(total_nvx):
    result_slow.append(geodesic_distance_wrapper(i))

# are they the same?
for i  in range(total_nvx):
    print(result_slow[i]==result_fast[i])


for i in range(20):
    print(true_result[i]==result[i])
#     print(np.round(true_result[i], 3))
#     print(np.round(result[i],3))
# # plt.imshow(np.array(true_result))
# print(np.array(true_result))
# print('\n\n\n\n\n')
# print(np.array(result))


In [ ]:

fs_dir = os.environ["SUBJECTS_DIR"]


In [ ]:
dir(gdist.surf['lh'])


In [ ]:
cprf_obj['L'].pd_params['cmf'] = cmf['full']

In [ ]:
cprf_obj['L'].rapid_p_corr(
    px='ecc', py='cmf', th={'min-rsq':.1, 'min-cmf':0, 'roi':roi_mask}, do_line=True
)

In [ ]:
distX = gdist.get_ivx_dist(300)
distX[300] = 50
distX_mask = np.array(np.isnan(distX)==0, dtype=bool)


In [ ]:
gdist.get_ivx_dist_closest(207)

In [ ]:
gdist.n_vx

In [ ]:
def get_CMF_LONG(prf_obj, gdist_obj, th):
    vx_mask = prf_obj.return_vx_mask(th)
    vx_mask_hemi = {}
    vx_mask_hemi['lh'] = vx_mask[:gdist_obj.n_vx['lh']]
    vx_mask_hemi['rh'] = vx_mask[gdist_obj.n_vx['lh']:]
    cmf = np.zeros(prf_obj.n_vox) * np.nan
    cmf = {}
    cmf['lh'] = np.zeros(gdist_obj.n_vx['lh'])
    cmf['rh'] = np.zeros(gdist_obj.n_vx['rh'])
    for hemi in ['lh', 'rh']:        
        vx_mask_ivx = np.where(vx_mask_hemi[hemi])[0]
        for ivx in vx_mask_ivx[:5]:
            ivx_dist = gdist.lh_surf.geodesic_distance(ivx)

    for ivx in vx_mask_ivx[:5]:
        ivx_dist = gdist.lh_surf.geodesic_distance(ivx)
        
        close_ivx_mask = vx_mask[close_ivx]
        close_ivx = close_ivx[close_ivx_mask]
        close_val = close_val[close_ivx_mask]
        if close_ivx_mask.sum()<6: # Only continue if enough neighbours are present...
            continue

        tx,ty = prf_obj.pd_params['x'][ivx],prf_obj.pd_params['y'][ivx]
        sx,sy = prf_obj.pd_params['x'][close_ivx],prf_obj.pd_params['y'][close_ivx]
        
        mdiff_deg = np.mean(np.sqrt((tx-sx)**2 + (ty-sy)**2))
        mdiff_mm = np.mean(close_val)

        cmf[ivx] = mdiff_mm / mdiff_deg
    return cmf
    
cmf = get_CMF(
    prf_obj=cprf_obj['L'], 
    gdist_obj=gdist, 
    th={'min-rsq':.1, 'roi':roi_mask})

In [ ]:
def get_CMF(prf_obj, gdist_obj, th):
    vx_mask = prf_obj.return_vx_mask(th)
    vx_mask_ivx = np.where(vx_mask)[0]
    cmf = np.zeros(prf_obj.n_vox) * np.nan
    # for i in vx_mask_ivx:
    for ivx in vx_mask_ivx:
        if ivx not in gdist_obj.ivx_list:
            continue
        close_ivx, close_val = gdist_obj.get_ivx_dist_closest(ivx, closest_x=20)
        # Apply the vx mask (e.g., rsq threshold, to cortical distances)
        close_ivx_mask = vx_mask[close_ivx]
        close_ivx = close_ivx[close_ivx_mask]
        close_val = close_val[close_ivx_mask]
        if close_ivx_mask.sum()<6: # Only continue if enough neighbours are present...
            continue

        tx,ty = prf_obj.pd_params['x'][ivx],prf_obj.pd_params['y'][ivx]
        sx,sy = prf_obj.pd_params['x'][close_ivx],prf_obj.pd_params['y'][close_ivx]
        
        mdiff_deg = np.mean(np.sqrt((tx-sx)**2 + (ty-sy)**2))
        mdiff_mm = np.mean(close_val)

        cmf[ivx] = mdiff_mm / mdiff_deg
    return cmf
    
cmf = get_CMF(
    prf_obj=cprf_obj['L'], 
    gdist_obj=gdist, 
    th={'min-rsq':.1, 'roi':roi_mask})

In [ ]:
cprf_obj['L'].pd_params['cmf'] = cmf

In [ ]:

cprf_obj['L'].rapid_p_corr(px='ecc', py='cmf', th={'min-rsq':.1, 'roi':roi_mask, 'min-cmf':-1}, do_line=True)
# plt.hist(cprf_obj['L'].pd_params['cmf'])
plt.show()

In [ ]:
np.where(cprf_obj['L'].return_vx_mask({'min-rsq':.1}))

In [ ]:
fs = FSMaker(sub=sub,fs_dir=fs_dir)
th={'min-rsq':.1, 'roi':roi_mask, 'min-cmf':-1}
vx_mask = cprf_obj['L'].return_vx_mask(th={'min-rsq':.1, 'roi':roi_mask, 'min-cmf':-1})
this_data = cprf_obj['L'].pd_params['cmf'].to_numpy()
np.nanmin(this_data)
fs.add_surface(
    data = this_data,
    surf_name = f'{sub}-cmf',    
    vmin = 0, vmax=10, # min and max values of polar anlge 
    data_mask=vx_mask,
    cmap = 'magma', 
)
fs.open_fs_surface(fs.surf_list[0])

In [ ]:
surf_calc = lsopt.Neighbours(
    subject='sub-01', 
)
surf_calc.create_distance()